<h3> Sergio Andres Rios Gomez</h3>
<h4> EA4. MNIST desde cero </h4>

In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import struct
import os

<p> Funciones para leer los archivos .ubyte:</p> 

In [8]:
def load_images(filename):
    with open(filename, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num, rows * cols)
        return images / 255.0  # Normalizar

def load_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

In [9]:
DATA_PATH = "../data/raw/"
X_train = load_images(DATA_PATH + "train-images.idx3-ubyte")
y_train = load_labels(DATA_PATH + "train-labels.idx1-ubyte")
X_test = load_images(DATA_PATH + "t10k-images.idx3-ubyte")
y_test = load_labels(DATA_PATH + "t10k-labels.idx1-ubyte")

<p> One-hot encoding de etiquetas </p>

In [10]:
def one_hot_encode(labels, num_classes=10):
    return np.eye(num_classes)[labels]

y_train_oh = one_hot_encode(y_train)
y_test_oh = one_hot_encode(y_test)

<p> Inicializar red neuronal. Red simple:</p>
<li> Capa de entrada: 784 (28x28)</li>
<li> Capa oculta: 64 </li>
<li> Capa de salida: 10 (dígitos) </li>

In [11]:
def init_params(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

<p> Funciones de activación y pérdida </p>

In [12]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def cross_entropy(y_pred, y_true):
    m = y_pred.shape[0]
    log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)])
    return np.sum(log_likelihood) / m

<p> Forward y backward propagation </p>

In [13]:
def forward(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward(X, y, Z1, A1, A2, W2):
    m = X.shape[0]
    dZ2 = A2 - y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2

<p> Entrenamiento </p>

In [14]:
def train(X, y, X_val, y_val, epochs=10, lr=0.1, hidden_size=64):
    input_size = X.shape[1]
    output_size = y.shape[1]

    W1, b1, W2, b2 = init_params(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward(X, W1, b1, W2, b2)
        loss = cross_entropy(A2, y)

        dW1, db1, dW2, db2 = backward(X, y, Z1, A1, A2, W2)

        W1 -= lr * dW1
        b1 -= lr * db1
        W2 -= lr * dW2
        b2 -= lr * db2

        if (epoch + 1) % 1 == 0:
            _, _, _, A2_test = forward(X_val, W1, b1, W2, b2)
            acc = np.mean(np.argmax(A2_test, axis=1) == np.argmax(y_val, axis=1))
            print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Accuracy: {acc:.4f}")

    return W1, b1, W2, b2

<p> Evaluación </p>

In [15]:
def evaluate(X, y, W1, b1, W2, b2):
    _, _, _, A2 = forward(X, W1, b1, W2, b2)
    pred_labels = np.argmax(A2, axis=1)
    true_labels = np.argmax(y, axis=1)
    accuracy = np.mean(pred_labels == true_labels)
    return accuracy

<p> Entrenamiento y pruebas</p>

In [16]:
W1, b1, W2, b2 = train(X_train, y_train_oh, X_test, y_test_oh, epochs=20, lr=0.5, hidden_size=64)

test_acc = evaluate(X_test, y_test_oh, W1, b1, W2, b2)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1, Loss: 2.3016, Accuracy: 0.1135
Epoch 2, Loss: 2.3006, Accuracy: 0.1135
Epoch 3, Loss: 2.3002, Accuracy: 0.1135
Epoch 4, Loss: 2.2997, Accuracy: 0.1135
Epoch 5, Loss: 2.2993, Accuracy: 0.1135
Epoch 6, Loss: 2.2988, Accuracy: 0.1135
Epoch 7, Loss: 2.2983, Accuracy: 0.1135
Epoch 8, Loss: 2.2978, Accuracy: 0.1135
Epoch 9, Loss: 2.2972, Accuracy: 0.1135
Epoch 10, Loss: 2.2965, Accuracy: 0.1135
Epoch 11, Loss: 2.2959, Accuracy: 0.1135
Epoch 12, Loss: 2.2951, Accuracy: 0.1135
Epoch 13, Loss: 2.2943, Accuracy: 0.1135
Epoch 14, Loss: 2.2934, Accuracy: 0.1135
Epoch 15, Loss: 2.2924, Accuracy: 0.1135
Epoch 16, Loss: 2.2912, Accuracy: 0.1135
Epoch 17, Loss: 2.2900, Accuracy: 0.1135
Epoch 18, Loss: 2.2886, Accuracy: 0.1135
Epoch 19, Loss: 2.2870, Accuracy: 0.1135
Epoch 20, Loss: 2.2853, Accuracy: 0.1138
Test accuracy: 0.1138


<h3> Conclusiones</h3>
<li> Una epoch es una pasada completa por todos los datos de entrenamiento. </li>
<li>El loss o pérdida es un número que indica qué tan mal lo está haciendo el modelo. Un valor alto indica que el modelo se está equivocando mucho.</li>
<li>La accuracy es el porcentaje de predicciones correctas.</li>
<h3> ¿Qué hicimos en este proyecto?</h3>
<li>Cargar manualmente los datos del MNIST en formato .ubyte</li>
<li>Preprocesar los datos (aplanarlos, normalizarlos, codificar etiquetas).</li>
<li>Implementar una red neuronal básica de 2 capas, usando. Propagación hacia adelante, Funciones de activación, One-hot encoding, Cálculo de pérdida y accuracy</li>
<li>Entrenar la red por varias epochs</li>
<li>Evaluar el rendimiento del modelo en el conjunto de prueba</li>
<h3> ¿Por qué es importante hacer esto manualmente? </h3>
<li>Comprensión profunda. Entender cómo fluye la información en una red neuronal.</li>
<li>Valor como profesional. Diferencia alguien que entiende la teoría y la práctica.</li>
<li>Desarrollo de habilidades sólidas. Mejora la lógica, pensamiento algorítmico y habilidad con vectores y matrices.</li>